In [ ]:
# Basic dependencies

import os 
import os.path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append("/home/adacomp/CLASP")

print(os.environ['OPENAI_API_KEY']) 


In [ ]:
from utils.process import resize

img_size = 480
image_path = "/home/adacomp/CLASP/object_template/dress_1.png"
obs_image= resize(Image.open(image_path).convert('RGB'), target_res=img_size, resize=True, to_pil=True)

background_image_path = "/home/adacomp/CLASP/semantic_keypoints/prototype/background.png"
background = resize(Image.open(background_image_path).convert('RGB'), target_res=img_size, resize=True, to_pil=True)

In [ ]:
from cloud_services.apis.owlv2 import OWLViT
from cloud_services.apis.sam import SAM,visualize_image

detector = OWLViT()
sam = SAM()        

detected_objects = detector.detect_objects(
        image=obs_image,
        text_queries= ['dress','cloth', 'clothes', 'trousers', 'skirt', 'tshirt', 'towel', 'traiangle'],   #'white object'], #['cloth', 'trousers', 'skirt', 'tshirt', ],
        bbox_score_top_k=25,
        bbox_conf_threshold=0.10)

best_score = 0
best_boxes = None
for det in detected_objects:
    if det["score"] > best_score:
        best_score = det["score"]
        best_boxes = det['bbox']
if best_boxes is None:
    print("No clothes detected")
    exit(0)
    
        # print(best_boxes)
# visualize_image(obs_image, bboxes=[best_boxes], show=True, return_img=True)
masks = sam.segment_by_bboxes(image=obs_image, bboxes=[best_boxes])
print(np.array(masks[0]["segmentation"]).astype(np.uint8))
print(np.array(masks[0]["segmentation"].shape))
np.save("/home/adacomp/CLASP/object_template/dress_1.npy", np.array(masks[0]["segmentation"]).astype(np.uint8))
visualize_image(obs_image, masks=[mask["segmentation"] for mask in masks], show=True, return_img=True)     

In [ ]:
from utils.process import center_object
img_processed, mask_processed, _ = center_object(np.array(obs_image), np.array(masks[0]["segmentation"]), background, size_ratio=1.2)
plt.imshow(img_processed)
plt.axis('off')
plt.show()
visualize_image(img_processed, masks=[mask_processed], show=True, return_img=False) 
print("center process")
obs_processed = Image.fromarray(img_processed)

In [ ]:
import copy
from skimage.morphology import medial_axis
from skimage.filters import gaussian
import cv2 as cv

def sample_uniform_keypoints(contour, num_samples):
    # Calculate the cumulative distances along the contour
    contour = contour.squeeze()  # Remove redundant dimension
    cumulative_distances = [0]
    
    for i in range(1, len(contour)):
        dist = np.linalg.norm(contour[i] - contour[i - 1])
        cumulative_distances.append(cumulative_distances[-1] + dist)
    
    total_length = cumulative_distances[-1]
    step_size = total_length / (num_samples - 1)  # Uniform step size for sampling
    print("step_size", step_size)
    
    sampled_points = [contour[0]]  # Start with the first point

    # Traverse the contour and sample points at uniform intervals
    current_distance = step_size
    for i in range(1, len(contour)):
        while cumulative_distances[i] >= current_distance:
            # Linear interpolation between two points
            t = (current_distance - cumulative_distances[i - 1]) / (cumulative_distances[i] - cumulative_distances[i - 1])
            sampled_point = (1 - t) * contour[i - 1] + t * contour[i]
            sampled_points.append(sampled_point)
            current_distance += step_size

    return np.array(sampled_points, dtype=np.int32)

mask = copy.deepcopy(mask_processed)

#  Apply Gaussian blur (sigma controls the amount of smoothing)
smoothed_image = gaussian(mask, sigma=1)

# Threshold back to binary
mask_cleaned = smoothed_image > 0.5
skeleton, distance = medial_axis(mask,return_distance= True)                      

# for contour in contours:
#     for point in contour:
#         cv.circle( self.draw_depth , tuple(point[0]), 3, (0, 0, 255), -1)  # Red keypoints
contours, _ = cv.findContours(mask.astype(np.uint8)*255, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv.contourArea)
contour_index = sample_uniform_keypoints(contour, num_samples=50)
print("contour_shape", contour_index.shape)

In [ ]:
from utils.geometric_utils import detect_robust_direction_changes, remove_overlapping_keypoints, fps

inflection_points = detect_robust_direction_changes(contour_index, angle_threshold=5)
skeleton_points = np.argwhere(skeleton)
skeleton_index = sample_uniform_keypoints(skeleton_points, num_samples=50)
candidate_skeleton_keypoints = fps(skeleton_points, 10)

all_points = np.concatenate((contour_index, fps(skeleton_points[:,::-1],40)), axis=0)
all_points = remove_overlapping_keypoints(all_points,min_distance=20)
candidate_keypoints = fps(all_points, 15)

In [ ]:
from utils.visual_prompt_utils import annotate_candidate_keypoints

#annotated_image_raw = annotate_candidate_keypoints(obs_image, all_points, add_caption=False)


annotated_image = annotate_candidate_keypoints(obs_processed, candidate_keypoints, add_caption=True)
plt.imshow(annotated_image)
plt.axis('off')
plt.show()

In [ ]:
from utils.gpt_utils import load_prompts
    
prompts = load_prompts()
prompt_kp = prompts['semantic_kp']
print(prompt_kp)

In [ ]:
from utils.gpt_utils import request_gpt

res = request_gpt("",[annotated_image],prompt_kp)
print(res)
plt.imshow(annotated_image)
plt.axis('off')
plt.show()